## Finding the Best Way to Swap Coins

In [1]:
import pandas as pd
from IPython.core.display import display, HTML

from src.data_extractors import get_pools, get_prices
from config import BOSTROM_NODE_URL, POOL_FEE

source_coin = 'boot'
target_coin = 'hydrogen'
source_amount = 1_000_000_000
max_slippage = 0.15

#### Get Bostrom and Osmosis Pools Data

In [2]:
pools_df = get_pools(display_data=True)

network,id,type_id,balances,reserve_coin_denoms,swap_fee
bostrom,1,1,"[{'denom': 'boot', 'amount': '72029970248'}, {'...","[boot, hydrogen]",0.003
bostrom,2,1,"[{'denom': 'boot', 'amount': '17622345734'}, {'...","[boot, milliampere]",0.003
bostrom,3,1,"[{'denom': 'boot', 'amount': '22031676253'}, {'...","[boot, tocyb]",0.003
bostrom,4,1,"[{'denom': 'hydrogen', 'amount': '65774354748'}...","[hydrogen, tocyb]",0.003
bostrom,5,1,"[{'denom': 'hydrogen', 'amount': '446376650534'...","[hydrogen, milliampere]",0.003
bostrom,6,1,"[{'denom': 'hydrogen', 'amount': '492267497973'...","[hydrogen, millivolt]",0.003
bostrom,7,1,"[{'denom': 'hydrogen', 'amount': '21173918789'}...","[hydrogen, uosmo in bostrom]",0.003
bostrom,8,1,"[{'denom': 'hydrogen', 'amount': '35730395079'}...","[hydrogen, uatom in bostrom]",0.003
bostrom,9,1,"[{'denom': 'uosmo in bostrom', 'amount': '46638...","[uosmo in bostrom, uatom in bostrom]",0.003
bostrom,10,1,"[{'denom': 'milliampere', 'amount': '8835650'},...","[milliampere, millivolt]",0.003


#### Calculate Prices

In [3]:
price_df = get_prices(pools_df=pools_df, display_data=True)

,uatom in osmosis,tocyb,hydrogen,millivolt,uatom in bostrom,boot in osmosis,boot,uosmo,milliampere,uosmo in bostrom
uatom in osmosis,1,NaN,NaN,NaN,1,0.010123,NaN,0.285648,NaN,NaN
tocyb,NaN,1,0.269553,NaN,NaN,NaN,0.515195,NaN,NaN,NaN
hydrogen,NaN,3.687613,1,128780.897106,217.100749,NaN,1.881198,NaN,17228.260101,54.423715
millivolt,NaN,NaN,0.000008,1,NaN,NaN,NaN,NaN,0.131626,NaN
uatom in bostrom,1,NaN,0.004579,NaN,1,NaN,NaN,NaN,NaN,0.252945
boot in osmosis,98.197007,NaN,NaN,NaN,NaN,1,1,27.982193,NaN,NaN
boot,NaN,1.929385,0.528391,NaN,NaN,1,1,NaN,9060.369375,NaN
uosmo,3.47984,NaN,NaN,NaN,NaN,0.035523,NaN,1,NaN,1
milliampere,NaN,NaN,0.000058,7.551792,NaN,NaN,0.00011,NaN,1,NaN
uosmo in bostrom,NaN,NaN,0.018264,NaN,3.929746,NaN,NaN,1,NaN,1


#### Search the best way to swap coins

In [4]:
# main: coin_source -> coin_target
ways = []
pool_main_df = pools_df[(pools_df.reserve_coin_denoms.isin([[source_coin, target_coin]])) | (pools_df.reserve_coin_denoms.isin([[target_coin, source_coin]]))]
if len(pool_main_df) > 0:
    ways = [[[source_coin, target_coin]]]

# alternative: coin_source -> coin3 -> coin_target
coin3_list = list(price_df[(~price_df[source_coin].isna()) & (~price_df[target_coin].isna()) & (~price_df.index.isin([source_coin, target_coin]))].index)
for coin3 in coin3_list:
    ways.append([[source_coin, coin3], [coin3, target_coin]])

def way_to_str(way: list):
    return ", ".join(f"{way_item[0]} -> {way_item[1]}" for way_item in way)

print(f'ways count {len(ways)}')
print(''.join(f'{str(i + 1)}. {way_to_str(way)}\n' for i, way in enumerate(ways)))

ways count 3
1. boot -> hydrogen
2. boot -> tocyb, tocyb -> hydrogen
3. boot -> milliampere, milliampere -> hydrogen



In [5]:
def get_balance_by_coin(pool_balances: list, coin: str) -> int:
    try:
        return [float(item['amount']) for item in pool_balances if item['denom'] == coin][0]
    except Exception as e:
        print(pool_balances, coin, e)

def generate_swap_query(coin_from_amount: float,
                        coin_from: str,
                        coin_to: str,
                        coins_pool_df: pd.DataFrame,
                        price_df: pd.DataFrame = price_df,
                        max_slippage: float = max_slippage,
                        wallet: str = '$WALLET',
                        chain_id: str = 'bostrom',
                        node = BOSTROM_NODE_URL) -> str:

    _pool_id = coins_pool_df.loc[:, 'id'].to_list()[0]
    _pool_type = coins_pool_df.loc[:, 'type_id'].to_list()[0]
    _price = price_df.loc[coin_to, coin_from] * (1 + max_slippage)
    return f'cyber tx liquidity swap {_pool_id} {_pool_type} {int(coin_from_amount)}{coin_from} {coin_to} {_price:.6f} 0.003 --from {wallet} ' \
           f'--chain-id {chain_id} --gas 200000 --gas-prices 0.01boot --yes --node {node} --broadcast-mode block'


def calculate_swap(way: list,
                   coin1_amount: float,
                   pools_df: pd.DataFrame = pools_df) -> [float, list]:
    _coin_from_amount = coin1_amount
    coin2_way_queries = []
    for way_item in way:
        _coin_from = way_item[0]
        _coin_to = way_item[1]
        _coins_pool_df = pools_df[(pools_df.reserve_coin_denoms.isin([[_coin_from, _coin_to]])) | (pools_df.reserve_coin_denoms.isin([[_coin_to, _coin_from]]))]
        _coin_from_pool_amount = get_balance_by_coin(_coins_pool_df.balances.values[0], _coin_from)
        _coin_to_pool_amount = get_balance_by_coin(_coins_pool_df.balances.values[0], _coin_to)
        _coin_to_amount = _coin_from_amount * _coin_to_pool_amount / (_coin_from_pool_amount + 2 * _coin_from_amount) * (1 - POOL_FEE)
        coin2_way_queries.append(generate_swap_query(coin_from_amount=_coin_from_amount, coin_from=_coin_from, coin_to=_coin_to, coins_pool_df=_coins_pool_df))
        _coin_from_amount = _coin_to_amount
    coin2_way_amount = _coin_from_amount
    return coin2_way_amount, coin2_way_queries


def calculate_all_swap(amount_ways_list: list, print_message: bool = False) -> [float, list]:
    swap_amount = 0
    swap_queries = []
    for _way, _amount in amount_ways_list:
        _swap_amount_way, _swap_queries_way = calculate_swap(_way, coin1_amount=_amount)
        swap_amount += _swap_amount_way
        swap_queries.append(_swap_queries_way)
        if print_message:
            print(f'{_way}\n{calculate_swap(_way, coin1_amount=_amount):>,.1f}\n')
    return swap_amount, swap_queries


def get_best_swap(ways: list = ways, amount = source_amount,  number_points: int = 10, print_message: bool = False) -> [pd.DataFrame, float]:
    _amount_list = [[coef_1 / number_points * amount, coef_2 / number_points * amount, (number_points - coef_1 - coef_2) / number_points * amount]
                   for coef_1 in range(number_points + 1)
                   for coef_2 in range(number_points - coef_1 + 1)]
    _swap_amount_list = []
    for _amount_list_item in _amount_list:
        _amount_ways_list = [[_way, _amount] for _way, _amount in zip(ways, _amount_list_item)]
        _swap_result, _swap_queries = calculate_all_swap(_amount_ways_list, print_message=print_message)
        _swap_amount_list.append([_amount_list_item, _swap_result, _swap_queries])

    _swap_amount_df = pd.DataFrame(_swap_amount_list,
                                  columns = ['amount_by_way_item', 'swap_result', 'swap_queries'])\
                                  .sort_values('swap_result', ascending=False)
    if print_message:
        print(_swap_amount_df)
    _swap_max_amount_list = list(_swap_amount_df.loc[_swap_amount_df['swap_result'].idxmax()][['amount_by_way_item', 'swap_queries']])
    return pd.DataFrame(zip(ways, _swap_max_amount_list[0], _swap_max_amount_list[1]), columns=['way', 'amount', 'swap_queries']), \
        _swap_amount_df['swap_result'].max()

best_swap_df, max_result = get_best_swap(number_points=100)
display(HTML(best_swap_df.to_html(index=False, notebook=True, show_dimensions=False)))
print(f'Source amount {source_amount:>,} {source_coin}\nMax target amount {int(max_result):>,} {target_coin}\n')
for index, row in best_swap_df.iterrows():
    print(f'{way_to_str(row.way)}')
    print(f'Amount {int(row.amount):>,} {source_coin}')
    print('\n'.join(row.swap_queries), '\n')

way,amount,swap_queries
"[[boot, hydrogen]]",670000000.0,[cyber tx liquidity swap 1 1 670000000boot hydr...
"[[boot, tocyb], [tocyb, hydrogen]]",160000000.0,[cyber tx liquidity swap 3 1 160000000boot tocy...
"[[boot, milliampere], [milliampere, hydrogen]]",170000000.0,[cyber tx liquidity swap 2 1 170000000boot mill...


Source amount 1,000,000,000 boot
Max target amount 1,849,082,240 hydrogen

boot -> hydrogen
Amount 670,000,000 boot
cyber tx liquidity swap 1 1 670000000boot hydrogen 2.163378 0.003 --from $WALLET --chain-id bostrom --gas 200000 --gas-prices 0.01boot --yes --node https://rpc.bostrom.cybernode.ai:443 --broadcast-mode block 

boot -> tocyb, tocyb -> hydrogen
Amount 160,000,000 boot
cyber tx liquidity swap 3 1 160000000boot tocyb 0.592474 0.003 --from $WALLET --chain-id bostrom --gas 200000 --gas-prices 0.01boot --yes --node https://rpc.bostrom.cybernode.ai:443 --broadcast-mode block
cyber tx liquidity swap 4 1 81251043tocyb hydrogen 4.240755 0.003 --from $WALLET --chain-id bostrom --gas 200000 --gas-prices 0.01boot --yes --node https://rpc.bostrom.cybernode.ai:443 --broadcast-mode block 

boot -> milliampere, milliampere -> hydrogen
Amount 170,000,000 boot
cyber tx liquidity swap 2 1 170000000boot milliampere 0.000126 0.003 --from $WALLET --chain-id bostrom --gas 200000 --gas-prices 0.01